In [1]:
import pdfplumber
import re
import json
from utils import is_toc_page

In [10]:
path = '/Users/ngam/dev/text-clustering/data/senior-facilities-agreement.pdf'

In [11]:
raw_text = ''
with pdfplumber.open(path) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if not is_toc_page(text):
            raw_text += "\n" + text

In [12]:
print(raw_text)


EXECUTION VERSION
Dated 24 December 2021
SENIOR FACILITIES AGREEMENT
COBHAM ULTRA SENIORCO S.À R.L.
(as the Company)
arranged by
BARCLAYS BANK PLC
BNP PARIBAS SA
BNP PARIBAS FORTIS SA/NV
CREDIT SUISSE LOAN FUNDING LLC
CREDIT SUISSE INTERNATIONAL
HSBC BANK PLC
MORGAN STANLEY BANK INTERNATIONAL LIMITED
ROYAL BANK OF CANADA
JEFFRIES FINANCE LLC
LLOYDS BANK PLC
SUMITOMO MITSUI BANKING CORPORATION, LONDON BRANCH
GOLDMAN SACHS BANK USA
NATWEST MARKETS PLC
NATIONAL WESTMINSTER BANK PLC
UNICREDIT BANK AG
(as Mandated Lead Arrangers)
with
CREDIT SUISSE AG, CAYMAN ISLANDS BRANCH
(as Agent)
and
WILMINGTON TRUST (LONDON) LIMITED
(as Security Agent)
KIRKLAND & ELLIS INTERNATIONAL LLP
30 St. Mary Axe
London EC3A 8AF
Tel: +44 (0)20 7469 2000
Fax: +44 (0)20 7469 2001
www.kirkland.com
KE 39675-89:114878275.43
THIS AGREEMENT is dated 24 December 2021.
BETWEEN:
(1) COBHAM ULTRA SENIORCO S.À R.L., a private limited liability company (société à
responsabilité limitée) incorporated under the laws of Luxemb

In [29]:
def extract_clauses_from_text(text):
    # Patterns
    title_pattern = re.compile(r"^(\d{1,2})\.\s+(.+)$")
    subtitle_pattern = re.compile(r"^(\d{1,2}(?:\.\d{1,2})?)\s+([A-Z][^\n]+)$")
    subclause_pattern = re.compile(r"^\s*\(([a-zA-Z]|[ivxlcdm]+|\d+)\)\s+(.*?)(?=\n\s*\([a-zA-Zivxlcdm\d]+\)\s+|\Z)", re.IGNORECASE)
    definition_pattern = re.compile(r'^"([^"]+)"\s+means:?\s*$')

    current_title = None
    current_definition = None
    subclauses = []
    definitions = []
    subtitles = []
    clauses = [{
                "number": "",
                "title": "Cover page and Borrowers",
                "full_title": "",
                'text': '',
                'subtitles': [{
                    'number': "",
                    'subtitle':"no_subtitle",
                    'full_subtitle': "",
                    'subclauses': subclauses,
                    'text': '',
                    'definitions' : {}
                }]
            }]

    # Split text into lines
    lines = text.split("\n")

    for line in lines:
        line = line.strip()
        if not line:
            continue

        title_match = title_pattern.match(line)
        subtitle_match = subtitle_pattern.match(line)
        subclause_match = subclause_pattern.match(line)
        definition_match = definition_pattern.match(line)

        if title_match:
            subtitles = []
            subclauses = []
            current_definition = None
            current_title = {
                "number": title_match.group(1),
                "title": title_match.group(2).strip(),
                "full_title": line,
                'text': '',
                'subtitles': subtitles
            }
            clauses.append(current_title)

        elif subtitle_match and current_title:
            subclauses = []
            current_definition = None
            subtitles.append({
                'number': subtitle_match.group(1),
                'subtitle': subtitle_match.group(2),
                'full_subtitle': line,
                'subclauses': subclauses,
                'text': '',
                'definitions' : {}
            })
            clauses[-1]['subtitles'] = subtitles

        elif subclause_match and clauses[-1]['subtitles']:
            subclause = {
                "label": subclause_match.group(1),
                "text": subclause_match.group(2).strip(),
                "full_text": line
            }

            if current_definition:
                definitions.append(subclause)
                clauses[-1]['subtitles'][-1]['definitions'][current_definition] = definitions
            else:
                subclauses.append(subclause)
                clauses[-1]['subtitles'][-1]['subclauses'] = subclauses

        elif definition_match and clauses[-1]['subtitles']:
            current_definition = line
            definitions = []
            clauses[-1]['subtitles'][-1]['definitions'][current_definition] = []

        elif subclauses:
            clauses[-1]['subtitles'][-1]['subclauses'][-1]['text'] += "\n" + line

        elif definitions and clauses[-1]['subtitles'] and current_definition:
            clauses[-1]['subtitles'][-1]['definitions'][current_definition][-1]['text'] += "\n" + line

        elif subtitles:
            clauses[-1]['subtitles'][-1]['text'] += "\n" + line
        else:
            clauses[-1]['text'] += "\n" + line

    return clauses

In [30]:
structured_data = {"clauses": extract_clauses_from_text(raw_text)}
output_json_path = "output_clauses_v2.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(structured_data, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON saved to: {output_json_path}")



✅ JSON saved to: output_clauses_v2.json


In [32]:
# Load json file

with open(output_json_path, 'r') as f:
    data = json.load(f)

In [33]:
for index, clause in enumerate(data['clauses']):
    print(index, clause.keys())

0 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
1 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
2 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
3 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
4 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
5 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
6 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
7 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
8 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
9 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
10 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
11 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
12 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
13 dict_keys(['number', 'title', 'full_title', 'text', 'subtitles'])
14 dict_keys(['number', 'title', 'full_title

In [59]:
for subtitle in data['clauses'][31]['subtitles']:
    print(subtitle.keys())

dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])
dict_keys(['number', 'subtitle', 'full_subtitle', 'subclauses', 'text', 'definitions'])


In [2]:
# Store data into NEO4J database
from utils import get_database_driver

driver = get_database_driver()

In [34]:
delete_query = "MATCH (n) DETACH DELETE n"
driver.execute_query(delete_query)

/var/folders/8j/6df62lm96l3_w6q0mjw5h3b40000gn/T/ipykernel_37167/2769981186.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(delete_query)


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x395072650>, keys=[])

In [36]:
def create_graph(tx, clause):
    tx.run("""
        MERGE (t:Title {label: $full_title})
        SET t.number = $clause_number
        SET t.title = $clause_title
        SET t.text = $text

        WITH t
        UNWIND $subtitles AS subtitle
            MERGE (s:Subtitle {label: subtitle.full_subtitle})
            SET s.number = subtitle.number
            SET s.subtitle = subtitle.subtitle
            SET s.text = subtitle.text
            MERGE (t)-[:HAS_SUBTITLE]->(s)

            WITH s, subtitle
            UNWIND subtitle.subclauses AS subclause
                MERGE (sc:Subclause {label: subclause.label})
                SET sc.text = subclause.text
                SET sc.full_text = subclause.full_text
                MERGE (s)-[:HAS_SUBCLAUSE]->(sc)

            WITH s, subtitle
                FOREACH (def_key IN keys(subtitle.definitions) |
                    FOREACH (item IN subtitle.definitions[def_key] |
                        MERGE (d:Definition {term: def_key, label: item.label})
                        SET d.text = item.text
                        SET d.full_text = item.full_text
                        MERGE (s)-[:HAS_DEFINITION]->(d)
            )
            )
    """, clause_title=clause["title"],
         full_title=clause["full_title"],
         text=clause["text"],
         clause_number=clause["number"],
         subtitles=clause["subtitles"])

# Load into Neo4j
with driver.session() as session:
    for clause in data["clauses"]:
        session.write_transaction(create_graph, clause)

driver.close()

/var/folders/8j/6df62lm96l3_w6q0mjw5h3b40000gn/T/ipykernel_37167/1442444900.py:39: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/8j/6df62lm96l3_w6q0mjw5h3b40000gn/T/ipykernel_37167/1442444900.py:41: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, clause)


In [ ]:
# Create FULLTEXT INDEXES

"""CREATE FULLTEXT INDEX title_subtitle_labels IF NOT EXISTS
FOR (n:Title|Subtitle)
ON EACH [n.label, n.text]"""

#KEYWORD SEARCH
"""CALL db.index.fulltext.queryNodes("title_subtitle_labels", "Confidential, co") YIELD node, score
RETURN node.label, node.text, score """